## Tutorial demonstrating useful functionality of Incidence Analysis for model debugging
We will create a small system of equations representing solid phase
thermodynamics that was encountered when debugging a chemical 
looping model and use a maximum matching, the Dulmage-Mendelsohn
partition, and block triangularization to analyze what is wrong
with it.

### Imports
We import Pyomo, NumPy, Matplotlib, the incidence graph interface we will use to run graph algorithms, a function for getting an incidence matrix (in a particular order), and a function for creating the model used in this example.

In [ ]:
import pyomo.environ as pyo 
from pyomo.contrib.incidence_analysis import IncidenceGraphInterface
from pyomo.contrib.incidence_analysis.interface import (
    get_structural_incidence_matrix,
)
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
from pyomo.common.collections import ComponentSet

from idaes.core.util.model_statistics import degrees_of_freedom

import numpy as np
import matplotlib.pyplot as plt


# If you did not run setup.py, but are working from the directory
# where this file (and model.py) lives, use this import instead:
#from model import make_model
from incidence_examples.tutorial.model import make_model

### Make model
We check this model for zero degrees of freedom, and see if it
has a perfect matching of constraints and variables.

In [ ]:
m = make_model()

igraph = IncidenceGraphInterface(m)
matching = igraph.maximum_matching()
M = len(igraph.constraints)
N = len(igraph.variables)
print("Degrees of freedom = %s" % degrees_of_freedom(m))
print(M, N, len(matching))

print("\nMatching:")
# matching is a ComponentMap mapping constraints to variables
for con, var in matching.items():
    print()
    print(con.name)
    print(var.name)

### Display unmatched variables and constraints

In [ ]:
print("Unmatched constraints:")
for con in igraph.constraints:
    if con not in matching:
        print("  %s" % con.name)

print("Unmatched variables:")
matched_var_set = ComponentSet(matching.values())
unmatched_vars = []
for var in igraph.variables:
    if var not in matched_var_set:
        unmatched_vars.append(var)
        print("  %s" % var.name)

We now know what variables and constraints are unmatched.
If `sum_component_eqn` contained `flow_mass`, our system would
have a perfect matching.

### Plot an incidence matrix with matched variables and constraints on the diagonal

In [ ]:
# Need to put variables in the right order
matched_var_list = []
for con in igraph.constraints:
    if con in matching:
        matched_var_list.append(matching[con])
    else:
        # "Associate" this constraint with a random unmatched var.
        matched_var_list.append(unmatched_vars.pop())
incidence_matrix = get_structural_incidence_matrix(
    matched_var_list, igraph.constraints
)
plt.figure()
plt.spy(incidence_matrix)
plt.title("Matching on diagonal, initial")
plt.show()

Note that this matrix does not have a complete diagonal,
and cannot be permuted to have a complete diagonal.
This is what is meant by "structurally singular."

### Use Dulmage-Mendelsohn to investigate source of structural singularity
We may already see a potential fix, but it is always a good idea
to use the Dulmage-Mendelsohn partition when debugging a structually
singular system.

In [ ]:
var_dmp, con_dmp = igraph.dulmage_mendelsohn()
# The returned types here are named tuples, each with the fields
# "unmatched", "underconstrained", "overconstrained", and "square".
# Each entry in the tuples is a list of variables or constraints.
# The unmatched variables and underconstrained variables/constraints
# form the "underconstrained subsystem," which has too few
# constraints to solve for its variables, while the unmatched
# constraints and overconstrained variables/constraints form the
# "overconstrained subsystem," which has too many constraints.

print(len(var_dmp.unmatched))
print(len(con_dmp.underconstrained), len(var_dmp.underconstrained))
print(len(con_dmp.square), len(var_dmp.square))
print(len(con_dmp.overconstrained), len(var_dmp.overconstrained))
print(len(con_dmp.unmatched))

We have a 7-by-8 underconstrained system, a 5-by-5 square system,
and a 6-by-5 overconstrained system. The Dulmage-Mendelsohn 
partition tells us what variables and constraints are in these
systems.

In [ ]:
print('unmatched variables:')
for var in var_dmp.unmatched:
    print("  %s" % var.name)
print('underconstrained variables:')
for var in var_dmp.underconstrained:
    print("  %s" % var.name)
print('underconstraining constraints:')
for con in con_dmp.underconstrained:
    print("  %s" % con.name)

print('unmatched constraints:')
for con in con_dmp.unmatched:
    print("  %s" % con.name)
print('overconstrained variables:')
for var in var_dmp.overconstrained:
    print("  %s" % var.name)
print('overconstraining constraints:')
for con in con_dmp.overconstrained:
    print("  %s" % con.name)

Any variable in the underconstrained subsystem _could have been_
unmatched, as could any constraint in the overconstrained
subsystem. This partition tells us that we are over-specifying
density and under-specifying flow rate.

### Plot incidence matrix in Dulmage-Mendelsohn order

In [ ]:
variables = (
    var_dmp.unmatched
    + var_dmp.underconstrained
    + var_dmp.square
    + var_dmp.overconstrained
)
constraints = (
    con_dmp.underconstrained
    + con_dmp.square
    + con_dmp.overconstrained
    + con_dmp.unmatched
)
incidence_matrix = get_structural_incidence_matrix(variables, constraints)
plt.figure()
plt.spy(incidence_matrix)
plt.title("Dulmage-Mendelsohn ordering")
plt.show()

### Use knowledge from maximum matching and Dulmage-Mendelsohn to apply a structual fix
Our unmatched variable is `flow_mass` and our unmatched constraint
is `sum_component_eqn`, which sets the sum of mass fractions equal
to one. If we just wrote this as a sum of component flow rates
(equal to total flow rate `flow_mass`) instead of a sum of mass
fractions, our system would be structually nonsingular.

In [ ]:
m.sum_component_eqn.deactivate()

@m.Constraint()
def sum_flow_eqn(m):
    return (
        sum(m.flow_mass_comp[j] for j in m.component_list)
        == m.flow_mass
    )

### Re-check model for degrees of freedom and structual singularity

In [ ]:
# Re-construct IncidenceGraphInterface to capture changes
# made to the model.
igraph = IncidenceGraphInterface(m)
matching = igraph.maximum_matching()
M = len(igraph.constraints)
N = len(igraph.variables)
print("Degrees of freedom = %s" % degrees_of_freedom(m))
print(M, N, len(matching))

print("\nMatching:")
for con, var in matching.items():
    print()
    print(con.name)
    print(var.name)

The system is still square, and now has a perfect matching of
variables and constraints, so it is no longer structually
singular!

### Plot incidence matrix, now with zero-free diagonal

In [ ]:
matched_var_list = [matching[con] for con in igraph.constraints]
incidence_matrix = get_structural_incidence_matrix(
    matched_var_list, igraph.constraints
)
plt.figure()
plt.spy(incidence_matrix)
plt.title("Matching on diagonal, after fix")
plt.show()

### Check Jacobian for "numeric" nonsingularity

In [ ]:
# PyNumero requires exactly one active objective
m._obj = pyo.Objective(expr=0.0)
nlp = PyomoNLP(m)
jacobian = nlp.evaluate_jacobian()
print("Condition number = %1.2e" % np.linalg.cond(jacobian.toarray()))

This seems pretty "numerically singular"...

### Use a block triangular decomposition to try to determine where the singularity is coming from

In [ ]:
var_blocks, con_blocks = igraph.get_diagonal_blocks()
# Return type: two lists of lists
# These are the diagonal blocks in a block triangularization of
# the incidence graph/Jacobian. These also happen to be the
# strongly connected components of a directed graph of variable/
# constraint dependence. The directed graph is not unique,
# but the strongly connected components (and these diagonal blocks)
# are.

for i, (vars, cons) in enumerate(zip(var_blocks, con_blocks)):
    dim = len(vars)
    print("Block %s, dim = %s" % (i, dim))
    submatrix = nlp.extract_submatrix_jacobian(vars, cons)
    cond = np.linalg.cond(submatrix.toarray())
    print("Condition number = %1.2e" % cond)
    print("  Variables:")
    for var in vars:
        print("    %s" % var.name)
    print("  Constraints:")
    for con in cons:
        print("    %s" % con.name)

The singularity appears to be coming from Block 1, which contains
the `sum_flow_eqn` we just added. This is because when the
sum of component mass fractions equals one, `sum_flow_eqn` is a 
linear combination of the other three equations in this block.

### Plot the incidence matrix in block triangular form

In [ ]:
variables = sum(var_blocks, [])
constraints = sum(con_blocks, [])
incidence_matrix = get_structural_incidence_matrix(variables, constraints)
plt.figure()
plt.spy(incidence_matrix)
plt.title("Block triangular permutation")
plt.show()

### Go back to the drawing board and come up with a real fix
As the Dulmage-Mendelsohn partition has shown us, we are
"underconstraining flow rate" and "overconstraining density."
But there should be a relationship between these quantities
that we are not including here. Particle density will be determined
by the flow rate, and skeletal density will be determined by the
composition, so particle porosity must be allowed to vary to avoid
overspecifying one of these variables. (Or we could relax the
requiremenet that skeletal density is related to skeletal
density of individual components...)

In [ ]:
m.sum_flow_eqn.deactivate()
m.sum_component_eqn.activate()
m.particle_porosity.unfix()

# Need an equation to relate density and flow rate
@m.Constraint()
def flow_density_eqn(b):
    return m.flow_mass == m.velocity * m.area * m.dens_mass_particle

### Make sure we still have zero degrees of freedom and a perfect matching

In [ ]:
print(degrees_of_freedom(m))
# Re-construct IncidenceGraphInterface to capture change in model
igraph = IncidenceGraphInterface(m)
matching = igraph.maximum_matching()
M = len(igraph.constraints)
N = len(igraph.variables)
print(M, N, len(matching))

### Check numeric singularity

In [ ]:
nlp = PyomoNLP(m)
jacobian = nlp.evaluate_jacobian()
cond = np.linalg.cond(jacobian.toarray())
print("Condition number = %1.2e" % cond)

Looks nonsingular! For good measure, perform a block
triangularization and get the condition number of each diagonal
block.

In [ ]:
var_blocks, con_blocks = igraph.get_diagonal_blocks()
for i, (vars, cons) in enumerate(zip(var_blocks, con_blocks)):
    dim = len(vars)
    print("Block %s, dim = %s" % (i, dim))
    submatrix = nlp.extract_submatrix_jacobian(vars, cons)
    cond = np.linalg.cond(submatrix.toarray())
    print("Condition number = %1.2e" % cond)
    print("  Variables:")
    for var in vars:
        print("    %s" % var.name)
    print("  Constraints:")
    for con in cons:
        print("    %s" % con.name)

Looks good! We even have fewer diagonal blocks of dimension greater
than one now.

In [ ]:
variables = sum(var_blocks, [])
constraints = sum(con_blocks, [])
incidence_matrix = get_structural_incidence_matrix(variables, constraints)
plt.figure()
plt.spy(incidence_matrix)
plt.title("Block triangular permutation, after fix")
plt.show()